In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
from config import g_key

In [2]:
weather_data = pd.read_csv("../python-api-challenge/data_output.csv")
weather_data

,Unnamed: 0,City,Temperature(F),Humidity(%),Cloudiness(%),Wind Speed(mph),Latitude,Longitude
0,0,namibe,82.40,47.0,20.0,6.91,-15.1961,12.1522
1,1,butaritari,81.09,78.0,85.0,17.92,3.0707,172.7902
2,2,jamestown,35.24,80.0,1.0,11.50,42.0970,-79.2353
3,3,mahebourg,86.00,74.0,40.0,16.11,-20.4081,57.7000
4,4,iqaluit,-11.09,77.0,90.0,3.89,63.7506,-68.5145
...,...,...,...,...,...,...,...,...
627,627,geraldton,95.00,34.0,14.0,17.27,-28.7667,114.6000
628,628,namtsy,-56.20,58.0,90.0,3.56,62.7161,129.6658
629,629,cayenne,75.20,100.0,75.0,3.44,4.9333,-52.3333
630,630,tambura,98.24,12.0,0.0,3.18,5.6000,27.4667


In [3]:
weather_clean = weather_data.dropna(thresh=4)

In [4]:
gmaps.configure(api_key=g_key)

locations = weather_clean[["Latitude", "Longitude"]]

humidity = weather_clean["Humidity(%)"]

In [5]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
city_df = weather_clean.loc[(weather_clean["Wind Speed(mph)"] <= 10) & (weather_clean["Cloudiness(%)"] == 0) & \
                                   (weather_clean["Temperature(F)"] >= 70) & (weather_clean["Temperature(F)"] <= 80)].dropna()
city_df

,Unnamed: 0,City,Temperature(F),Humidity(%),Cloudiness(%),Wind Speed(mph),Latitude,Longitude
6,6,busselton,79.61,42.0,0.0,1.99,-33.6500,115.3333
43,43,mankera,70.93,19.0,0.0,3.24,31.3866,71.4407
60,60,tairua,70.00,69.0,0.0,10.00,-37.0167,175.8500
159,159,ulladulla,72.70,73.0,0.0,6.91,-35.3500,150.4667
220,220,san pedro del parana,73.62,83.0,0.0,6.91,-26.8333,-56.2000
247,247,tautira,78.40,78.0,0.0,3.44,-17.7333,-149.1500
286,286,tiarei,78.39,78.0,0.0,3.44,-17.5333,-149.3333
401,401,sao joao da barra,75.20,88.0,0.0,2.30,-21.6403,-41.0511
519,519,caravelas,77.83,82.0,0.0,5.93,-17.7125,-39.2481
552,552,yeppoon,78.80,61.0,0.0,8.05,-23.1333,150.7333


In [9]:
Hotel_df = city_df.loc[:,["City", "Latitude", "Longitude"]]

Hotel_df["Hotel Name"] = ""


Hotel_df

,City,Latitude,Longitude,Hotel Name
6,busselton,-33.6500,115.3333,
43,mankera,31.3866,71.4407,
60,tairua,-37.0167,175.8500,
159,ulladulla,-35.3500,150.4667,
220,san pedro del parana,-26.8333,-56.2000,
247,tautira,-17.7333,-149.1500,
286,tiarei,-17.5333,-149.3333,
401,sao joao da barra,-21.6403,-41.0511,
519,caravelas,-17.7125,-39.2481,
552,yeppoon,-23.1333,150.7333,


In [11]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [13]:
for index, row in Hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"

    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    
    results = response['results']
    
  
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        Hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    
    except (KeyError, IndexError):
        print("City not found... skipping.")
        
    print("------------")
    

print("-------End of Search-------")

Retrieving Results for Index 6: busselton.
Closest hotel in busselton is The Sebel Busselton.
------------
Retrieving Results for Index 43: mankera.
Closest hotel in mankera is Gohar wala.
------------
Retrieving Results for Index 60: tairua.
Closest hotel in tairua is Grand Mercure Puka Park Resort.
------------
Retrieving Results for Index 159: ulladulla.
Closest hotel in ulladulla is Beachhouse Mollymook.
------------
Retrieving Results for Index 220: san pedro del parana.
Closest hotel in san pedro del parana is Hotel Oasis.
------------
Retrieving Results for Index 247: tautira.
City not found... skipping.
------------
Retrieving Results for Index 286: tiarei.
Closest hotel in tiarei is Le Rocher de Tahiti.
------------
Retrieving Results for Index 401: sao joao da barra.
Closest hotel in sao joao da barra is OYO Pousada Mediterrâneo.
------------
Retrieving Results for Index 519: caravelas.
Closest hotel in caravelas is Mar Doce de Abrolhos.
------------
Retrieving Results for In

In [14]:
Hotel_df

,City,Latitude,Longitude,Hotel Name
6,busselton,-33.6500,115.3333,The Sebel Busselton
43,mankera,31.3866,71.4407,Gohar wala
60,tairua,-37.0167,175.8500,Grand Mercure Puka Park Resort
159,ulladulla,-35.3500,150.4667,Beachhouse Mollymook
220,san pedro del parana,-26.8333,-56.2000,Hotel Oasis
247,tautira,-17.7333,-149.1500,
286,tiarei,-17.5333,-149.3333,Le Rocher de Tahiti
401,sao joao da barra,-21.6403,-41.0511,OYO Pousada Mediterrâneo
519,caravelas,-17.7125,-39.2481,Mar Doce de Abrolhos
552,yeppoon,-23.1333,150.7333,Yeppoon Beach House


In [16]:
info_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""

hotel_info = [info_template.format(**row) for index, row in Hotel_df.iterrows()]
locations = Hotel_df[["Latitude", "Longitude"]]

In [18]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))